### TO DO ###

- [x] Fix no data for first week
- [x] Add cumulative to standings
- [x] Check by week when new data comes
- [ ] Add value for players who don't play
- [ ] Goal difference statistic
- [ ] Above or below average score
- [ ] Pandas Stylesheet
- [ ] Graphs

In [250]:
import requests
import pandas as pd
import numpy as np

#### Definitions ####

In [21]:
owners = ['Luke','Emma','Fat','Olivia','Claire','Matt','George']
owner_ids = ['18900','18935','18967','19180','19424','20043','20626']
team_details_url = {}
lineup_url = {}

for owner_id, owner in enumerate(owners):
    team_details_url[owner] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/history'
    for gameweek in range(1,39):
        lineup_url[owner,gameweek] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/event/'+str(gameweek)

bootstrap_url = 'https://draft.premierleague.com/api/bootstrap-static'
fantasy_bootstrap_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_owner_url = 'https://draft.premierleague.com/api/league/5857/element-status'
league_details_url = 'https://draft.premierleague.com/api/league/5857/details'

current_gameweek = requests.get(bootstrap_url).json()['events']['current']
total_fantasy_players = requests.get(fantasy_bootstrap_url).json()['total_players']

In [22]:
def url_to_df(url, datatype):
    request = requests.get(url)
    json = request.json()
    df = pd.DataFrame(json[datatype])
    return df

#### Create Dataframes ####

In [23]:
player_types_df = url_to_df(bootstrap_url,'element_types')

teams_df = url_to_df(bootstrap_url,'teams')

player_owner_df = url_to_df(player_owner_url,'element_status')
player_owner_df = player_owner_df.sort_values('element', ignore_index=True)

league_entries_df = url_to_df(league_details_url,'league_entries')
league_standings_df = url_to_df(league_details_url,'standings')
league_standings_df = league_standings_df.sort_values('league_entry', ignore_index=True)
league_standings_df = pd.concat([league_entries_df[['entry_id','id','entry_name','player_first_name','waiver_pick']], league_standings_df[['last_rank','rank','event_total','total']]], axis=1)

player_info_df = url_to_df(bootstrap_url,'elements')
player_info_df['team'] = player_info_df.team.map(teams_df.set_index('id').name)
player_info_df['element_type'] = player_info_df.element_type.map(player_types_df.set_index('id').singular_name)
player_info_df.insert(3,'owner',player_owner_df.owner.map(league_entries_df.set_index('entry_id').player_first_name))

#### Functions for making specific DataFrames ####

In [235]:
def get_lineup_info(owner, gameweek = current_gameweek, with_totals = True):
    # check valid gameweek:
    if gameweek > current_gameweek:
        print("Gameweek not yet played")
        return('ERROR')
    
    # create dataframe:
    info_columns = ['name','team','position']
    points_columns = ['minutes','goals_scored','assists','saves','goals_conceded',
                      'own_goals','yellow_cards','red_cards','total_points']
    lineup_df = pd.DataFrame(columns=info_columns + ['value', 'selected %'] + points_columns,
                             index = url_to_df(lineup_url[owner, gameweek],'picks')['element'])
    
    # all player data:
    lineup_df['name'] = lineup_df.index.map(player_info_df.set_index('id').second_name)
    lineup_df['team'] = lineup_df.index.map(player_info_df.set_index('id').team)
    lineup_df['position'] = lineup_df.index.map(player_info_df.set_index('id').element_type)
    
    # player by player data:
    for i, player in enumerate(lineup_df.index):
        for data in points_columns + ['value', 'selected %']:
            lineup_df.loc[lineup_df.index == player, data] = 0
        player_url = 'https://draft.premierleague.com/api/element-summary/'+str(player)
        player_df = url_to_df(player_url,'history')
        fantasy_player_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(player)+'/'
        fantasy_player_df = url_to_df(fantasy_player_url,'history')
        if not player_df.loc[player_df['event'] == gameweek].empty:
            lineup_df.loc[lineup_df.index == player, 'value'] = fantasy_player_df.loc[fantasy_player_df['round'] == gameweek]['value'].iloc[0]/10
            lineup_df.loc[lineup_df.index == player, 'selected %'] = fantasy_player_df.loc[fantasy_player_df['round'] == gameweek]['selected'].iloc[0]/total_fantasy_players*100
            for data in points_columns:
                lineup_df.loc[lineup_df.index == player, data] = player_df.loc[player_df['event'] == gameweek][data].iloc[0]
    
    if with_totals:
        # add totals row:
        totals_df = pd.DataFrame(index = ['totals (inc. bench)','totals'])
        for data in info_columns:
            totals_df.loc[:, data] = ''
        for data in ['value'] + points_columns:
            totals_df.loc['totals (inc. bench)', data] = lineup_df[data].sum()
            totals_df.loc['totals', data] = lineup_df.head(11)[data].sum()
        totals_df.loc['totals (inc. bench)', 'name'] = sum(lineup_df['minutes']>0)
        totals_df.loc['totals', 'name'] = sum(lineup_df.head(11)['minutes']>0)
        totals_df.loc['totals (inc. bench)', 'selected %'] = lineup_df['selected %'].mean()
        totals_df.loc['totals', 'selected %'] = lineup_df.head(11)['selected %'].mean()
        totals_df.loc['totals (inc. bench)', 'team'] = lineup_df['team'].nunique()
        totals_df.loc['totals', 'team'] = lineup_df.head(11)['team'].nunique()
        lineup_df = pd.concat([lineup_df, totals_df])
        
    # make reader friendly:
    lineup_df.index.name = None
    lineup_df = lineup_df.rename(columns={'goals_scored': 'goals scored', 'goals_conceded': 'goals conceded',
                                          'own_goals': 'own goals', 'yellow_cards': 'yellow cards',
                                          'red_cards': 'red cards', 'total_points': 'total points'})
    return(lineup_df)

In [243]:
get_lineup_info('Matt',2,False)

,name,team,position,value,selected %,minutes,goals scored,assists,saves,goals conceded,own goals,yellow cards,red cards,total points
96,Pope,Burnley,Goalkeeper,5.5,15.7976,90,0,0,3,4,0,0,0,1
255,Robertson,Liverpool,Defender,7,12.1816,90,0,0,0,0,0,0,0,6
313,Wan-Bissaka,Man Utd,Defender,5.5,13.0565,0,0,0,0,0,0,0,0,0
347,Stevens,Sheffield Utd,Defender,5.5,4.61878,0,0,0,0,0,0,0,0,0
272,De Bruyne,Man City,Midfielder,11.5,31.1135,0,0,0,0,0,0,0,0,0
231,Maddison,Leicester,Midfielder,7,1.62329,28,0,0,0,1,0,0,0,1
284,Foden,Man City,Midfielder,6.5,4.21698,0,0,0,0,0,0,0,0,0
468,Jota,Wolves,Midfielder,6.4,2.60413,0,0,0,0,0,0,0,0,0
137,Townsend,Crystal Palace,Midfielder,6,0.779668,90,1,0,0,1,0,0,0,9
224,Vardy,Leicester,Forward,10.1,23.8711,90,0,0,0,2,0,0,0,2


In [165]:
def get_league_standings(Cumulative=True):
    gameweek_range = range(1,current_gameweek+1)
    # create dataframe:
    league_standings_df = pd.DataFrame(columns = gameweek_range, index = owners)
    league_standings_df.insert(current_gameweek, 'total points', 0)
    team_info_df = {}
    for owner in owners:
        team_info_df[owner] = url_to_df(team_details_url[owner],'history')
        for week in gameweek_range:
            points = team_info_df[owner].loc[team_info_df[owner]['event'] == week,'points'].iloc[0]
            if week == 1 or not Cumulative:
                league_standings_df.loc[owner, week] = points
            else:
                league_standings_df.loc[owner, week] = league_standings_df.loc[owner, week-1] + points
        if Cumulative:
            league_standings_df.loc[owner, 'total points'] = league_standings_df.loc[owner, current_gameweek]
        else:
            league_standings_df.loc[owner, 'total points'] = league_standings_df.loc[owner, :].sum()
    
    league_standings_df = league_standings_df.sort_values('total points', ascending=False)    
    return(league_standings_df)

In [166]:
get_league_standings(True)

,1,2,total points
Emma,43,104,104
Claire,39,80,80
Luke,27,76,76
Olivia,31,75,75
Fat,48,63,63
George,26,54,54
Matt,22,47,47


#### Get all info (only run when new info to retrieve) ####

In [287]:
def get_all_lineups():
    all_lineups_df = {}
    for gameweek in range(1, current_gameweek+1):
        for owner in owners:
            all_lineups_df[owner, gameweek] = get_lineup_info(owner, gameweek, True)
    return(all_lineups_df)

In [288]:
all_lineups_df = get_all_lineups()

In [289]:
all_lineups_df['Emma',2]

,name,team,position,value,selected %,minutes,goals scored,assists,saves,goals conceded,own goals,yellow cards,red cards,total points
252,Ramses Becker,Liverpool,Goalkeeper,6,11.0563,90,0,0,3,0,0,0,0,14
62,Dunk,Brighton,Defender,5,7.66489,90,0,0,0,0,0,0,0,7
81,Tarkowski,Burnley,Defender,5.5,5.24265,0,0,0,0,0,0,0,0,0
323,Fernández,Newcastle,Defender,5,1.55041,90,0,0,0,3,0,0,0,1
254,Salah,Liverpool,Midfielder,12,35.8679,90,0,0,0,0,0,0,0,3
241,Milner,Liverpool,Midfielder,5.5,0.194427,26,0,0,0,0,0,0,0,1
113,Kanté,Chelsea,Midfielder,5,4.36349,90,0,0,0,2,0,0,0,2
193,Hernández Domínguez,Leeds,Midfielder,6,1.69148,0,0,0,0,0,0,0,0,0
166,de Andrade,Everton,Forward,8,10.9817,90,0,3,0,2,0,0,0,12
388,Kane,Spurs,Forward,10.5,12.5032,83,1,4,0,1,0,0,0,21


#### Parse All Lineups DF ####

In [370]:
def get_league_stats_df_by_gameweek(all_lineups_df, include_bench = False):
    if include_bench:
        value_to_get = 'totals (inc. bench)'
    else:
        value_to_get = 'totals'
    stats = ['name','team','value','selected %','minutes','goals scored','assists','saves','goals conceded','own goals','yellow cards','red cards','total points']
    league_stats_df = {}
    gameweek_range = range(1,current_gameweek+1)
    for gameweek in gameweek_range:
        league_stats_df[gameweek] = pd.DataFrame(columns = stats + ['league position'], index = owners)
        league_stats_df[gameweek].insert(5, 'productivity', 0)
    
        for owner in owners:
            for stat in stats:
                league_stats_df[gameweek].loc[owner, stat] = all_lineups_df[owner, gameweek].loc[value_to_get,stat]
        league_stats_df[gameweek] = league_stats_df[gameweek].sort_values('total points', ascending=False)
        for i, owner in enumerate(league_stats_df[gameweek].index):
            league_stats_df[gameweek].loc[owner, 'productivity'] = league_stats_df[gameweek].loc[owner, 'total points']/league_stats_df[gameweek].loc[owner, 'minutes']
            league_stats_df[gameweek].loc[owner, 'league position'] = (i+1)
        league_stats_df[gameweek] = league_stats_df[gameweek].rename(columns={'name': 'active players'})
    return(league_stats_df)

In [371]:
league_stats_df_by_gameweek = get_league_stats_df_by_gameweek(all_lineups_df)

In [381]:
league_stats_df_by_gameweek[1]

,active players,team,value,selected %,minutes,productivity,goals scored,assists,saves,goals conceded,own goals,yellow cards,red cards,total points,league position
Fat,11,8,70.5,7.37135,913,0.052574,3,1,3,11,0,0,0,48,1
Emma,10,9,69,7.75535,786,0.054707,3,0,0,14,0,0,0,43,2
Claire,10,9,76,5.45747,799,0.048811,2,3,0,13,0,2,0,39,3
Olivia,11,8,73,7.91703,953,0.032529,2,1,2,13,0,2,0,31,4
Luke,9,8,75,14.7561,610,0.044262,0,1,1,7,0,0,0,27,5
George,9,9,59,5.59126,685,0.037956,0,0,5,10,0,1,0,26,6
Matt,7,9,53.5,4.66025,509,0.043222,2,1,0,12,0,0,0,22,7


In [405]:
def get_league_stats_df_by_stat(league_stats_df_by_gameweek, stat, total_type = 'none', sort_by_week = 0):
    gameweek_range = range(1,current_gameweek+1)
    
    league_stats_df_by_stat = pd.DataFrame(columns = gameweek_range, index = owners)
    
    for owner in owners:
        for gameweek in gameweek_range:
            league_stats_df_by_stat.loc[owner, gameweek] = league_stats_df_by_gameweek[gameweek].loc[owner, stat]
        if total_type == 'mean' or total_type == 'average':
            league_stats_df_by_stat.loc[owner, 'average'] = league_stats_df_by_stat.loc[owner, gameweek_range].mean()
            league_stats_df_by_stat.sort_values('average', ascending=False)
        elif total_type == 'total':
            league_stats_df_by_stat.loc[owner, 'totals'] = league_stats_df_by_stat.loc[owner, gameweek_range].sum()
            league_stats_df_by_stat.sort_values('totals', ascending=False)

    if not sort_by_week == 0:
        league_stats_df_by_stat.sort_values(sort_by_week, ascending=False)
        
    return(league_stats_df_by_stat)

In [406]:
get_league_stats_df_by_stat(league_stats_df, 'selected %', 'mean')

,1,2,average
Luke,14.7561,16.1303,15.443159
Emma,7.75535,8.60593,8.180639
Fat,7.37135,9.5396,8.455475
Olivia,7.91703,10.4706,9.193802
Claire,5.45747,6.05974,5.758604
Matt,4.66025,10.6665,7.663390
George,5.59126,6.11588,5.853565


In [292]:
stats_df = get_league_stats_df_by_gameweek(all_lineups_df, 2, True)
stat_to_sort_by = 'selected %'
stats_df.sort_values(stat_to_sort_by, ascending=False)[[stat_to_sort_by,'total points','league position']]

,selected %,total points,league position
Luke,13.4506,51,3
Fat,9.82458,21,7
Matt,9.14359,34,5
Olivia,8.0695,47,4
Emma,7.21152,61,1
George,5.67814,33,6
Claire,5.24751,60,2
